In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from preprocess import load_dataset, featurisation
from sklearn.model_selection import train_test_split
from concrete import fhe

/Users/camilziane/.pyenv/versions/3.10.12/envs/hack_hf_zama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from concrete import fhe

# @fhe.compiler({"counter": "encrypted"})
# def increment(counter):
#    return (counter + 1) % 100

# print("Compiling `increment` function")
# increment_fhe = increment.compile(list(range(0, 100)), composable=True)

# print("Generating keyset ...")
# increment_fhe.keygen()

# print("Encrypting the initial counter value")
# counter = 0
# counter_enc = increment_fhe.encrypt(counter)

# print(f"| iteration || decrypted | cleartext |")
# for i in range(10):
#     counter_enc = increment_fhe.run(counter_enc)
#     counter = increment(counter)

#     # For demo purpose; no decryption is needed.
#     counter_dec = increment_fhe.decrypt(counter_enc)
#     print(f"|     {i}     || {counter_dec:<9} | {counter:<9} |")

In [4]:
# from concrete.ml.quantization import QuantizedArray
# X_train.shape
# X_train_q = QuantizedArray(8, X_train)
# X_train_q.qvalues,
# # np.max(X_train), np.min(X_train)

In [5]:
embeddings, labels = load_dataset(
    "./data/lfw_people/George_HW_Bush", cache=True)
# embeddings = featurisation(embeddings)
# standard scale the embeddings
embeddings = (embeddings - np.mean(embeddings, axis=0)) / np.std(embeddings, axis=0)
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

model = LogisticRegression(C=1/5)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Accuracy: {score}")
# recall compute
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)
recall = recall_score(y_test, y_pred, average='macro')
print(f"Recall: {recall}")

Accuracy: 0.9893617021276596
Recall: 0.8333333333333333


In [6]:
model.intercept_

array([-7.28123411])

In [52]:
from concrete.ml.torch.compile import compile_torch_model
import torch
import torch.nn as nn

class RegNet(nn.Module):
    def __init__(self, b):
        super().__init__()
        self.b = nn.Parameter(torch.ones(1)*b)  
    def forward(self, X, W): 
        # X = x[:, :128]
        # W = x[:, 128:]
        # X = x[0]
        # W = x[1]
        # print(X.shape, W.shape, self.b.shape)
        return ((X @ W.T + self.b) > 0).float() * 42

W = model.coef_
b = model.intercept_.reshape(-1, 1)
reg_net = RegNet(b)
X_stacked = np.hstack([X_train[0].reshape(1,-1), W])

In [53]:
W.shape, X_train[0].reshape(1,-1).shape

((1, 128), (1, 128))

In [54]:
# convert np array to torch tensor
W_torch = torch.tensor(W)
X_train0_torch = torch.tensor(X_train[0].reshape(1,-1))
W_torch.shape, X_train0_torch.shape

(torch.Size([1, 128]), torch.Size([1, 128]))

In [55]:
# reg_net((X_train0_torch,W_torch))

In [56]:
nb_sample = 100
X_train_rand = np.random.normal(0, 1, [nb_sample, X_train.shape[1]])
W_rand = np.random.normal(0, 1, [nb_sample, W.shape[1]])
X_rand_stack = np.hstack([X_train_rand, W_rand])

In [57]:
X_rand_stack.reshape(100,-1, 2).shape
X_rands = ((x[:,0].reshape(1,-1), x[:,1].reshape(1,-1)) for x in X_rand_stack.reshape(100,-1, 2))
X_rands = []

In [58]:
quantized_module = compile_torch_model(
    reg_net, # our model
    (X_train_rand, W_rand), # a representative input-set to be used for both quantization and compilation
    n_bits=6,
    rounding_threshold_bits={"n_bits": 6, "method": "approximate"}
)

In [59]:
# duplicate W and b to match the number of samples
W_test = np.repeat(W, X_test.shape[0], axis=0)
X_test_stacked = np.hstack([X_test, W_test])

In [60]:
mini_test = X_test_stacked[:1]
mini_test.shape

(1, 256)

In [61]:
# W_q=quantized_module.quantize_input(W_test)
# X_q=quantized_module.quantize_input(X_test)
# X_test_q=(quantized_module.quantize_input(np.hstack([X_test, W_test])))
# y_pred_q=quantized_module.quantized_forward(X_test_q, fhe="simulate")
# y_pred = quantized_module.dequantize_output(y_pred_q)

In [62]:
# y_pred = quantized_module.forward(mini_test, fhe="execute")

In [63]:
# y_pred_q

In [64]:
# y_pred.reshape(-1) - y_test

# Deployement

In [65]:
from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer

In [68]:
fhemodel_dev = FHEModelDev("./deploy", quantized_module)
fhemodel_dev.save()

In [69]:
fhemodel_client = FHEModelClient("./deploy", key_dir="./deploy")

In [70]:
evalution_key = fhemodel_client.get_serialized_evaluation_keys()

KeySetCache: miss, regenerating ./deploy/12078945368485275920


In [71]:
input_e = fhemodel_client.quantize_encrypt_serialize(X_test[[7]], W_test[[0]])

In [72]:
encrypted_prediction = FHEModelServer("./deploy").run(input_e, evalution_key)

In [99]:
encrypted_prediction

b'\x01\x00\x00\x00\x1d \x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x08\x00\x00\x00\x00\x00\x01\x00\x10\x80\x00\x00\x01\x00\x01\x00 \x80\x00\x00\x01\x00\x01\x00\x01\x00\x00\x00\x0e\x00\x00\x00\x01\x00\x00\x00B\x00\x08\x00\x00\x00\x00\xe8\xc0\x8b\xe0\xc8\x00\x00\x80\xab\x0ff\xa2\xdd\x00\x00\x80\xe9\xbc\x940\'\x00\x00\xbdvWu\xb6\x14\x00\x00pn\'\xe7/\xc7\x00\x00@gP"\xbb\xd6\x00\xc0\r|\xf1\x8e\xbd\xcd\x00\x00\xa0/\xcd\xbd\xd5\xe2\x00\x00\xfe\xbf\x8e\xf8\xa5\x07\x00\x00\xb0T*\xddb\xe0\x00\x00@\xa1-[Z\xd6\x00\x00\x90\xa1\xbf\xf9\xbe\x1d\x00\x00\x90\xd3>\xb7\xf7\x1d\x00\x00@\xa2\x9d\xb5\xbc\xfd\x00\x00\x80;\xb1\xdd\x0f\xbe\x00\x00\xb0\xe1\xce *\xf4\x00\x00\xc0r\x8a\x12b\xa2\x00\x00@\xa3\x8e\xab\xbb\xc3\x00\x00 %\xb5\xe8\x8eV\x00\x00\x00\xd2\xef\x9e\xaa\xa0\x00\x00\x10\xc7\xf98\xd4\xb7\x00\x00\xb0T\xaeD\x0fr\x00\x00\\B\xab%|\xec\x00\x00\xec)\xaf\xf0\xa1\x82\x00\x00`\xdb\xec\xa9\xb2\x95\x00\x00\xc0\x9e\x8f\x9f\x1d\x95\x00\x00`\x8fv\xa7\x8f8\x00\x00v\xf7}\xc7\xdf \x00

In [101]:
encrypted_prediction_magic = encrypted_prediction 

In [80]:
decrypted_prediction = fhemodel_client.deserialize_decrypt_dequantize(encrypted_prediction_magic)[0]

In [81]:
decrypted_prediction

array([42.])

In [37]:
y_test[7]

1